# PART 2: SQLite in Jupyter

## Overview
Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions. 



In [5]:
import sqlite3
import pandas as pd

# Connect to the file you see in your sidebar
con = sqlite3.connect('sqlite_db_pythonsqlite.db')



### Q10: Facilities with total revenue less than 1000

In [6]:
# Q10: Facilities with total revenue less than 1000, accounting for Member/Guest costs
query = """
SELECT name, total_revenue
FROM (
    SELECT f.name, 
           SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                    ELSE f.membercost * b.slots 
               END) AS total_revenue
    FROM Bookings AS b
    INNER JOIN Facilities AS f ON b.facid = f.facid
    GROUP BY f.name
) AS subquery
WHERE total_revenue < 1000
ORDER BY total_revenue;
"""

df_q10 = pd.read_sql_query(query, con)
print(df_q10)


            name  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


### Q11: Report of members and who recommended them in alphabetical order

In [7]:
# Q11: Report of members and who recommended them in alphabetical order
query = """
SELECT 
    m.surname AS member_surname, 
    m.firstname AS member_firstname, 
    r.surname AS rec_surname, 
    r.firstname AS rec_firstname
FROM Members AS m
INNER JOIN Members AS r ON m.recommendedby = r.memid
ORDER BY m.surname, m.firstname;
"""

df_q11 = pd.read_sql_query(query, con)
print(df_q11)

       member_surname member_firstname rec_surname rec_firstname
0               Bader         Florence    Stibbons        Ponder
1               Baker             Anne    Stibbons        Ponder
2               Baker          Timothy     Farrell        Jemima
3              Boothe              Tim      Rownam           Tim
4             Butters           Gerald       Smith        Darren
5              Coplin             Joan       Baker       Timothy
6             Crumpet            Erica       Smith         Tracy
7                Dare            Nancy    Joplette        Janice
8             Genting          Matthew     Butters        Gerald
9                Hunt             John     Purview     Millicent
10              Jones            David    Joplette        Janice
11              Jones          Douglas       Jones         David
12           Joplette           Janice       Smith        Darren
13          Mackenzie             Anna       Smith        Darren
14               Owen    

### Q12: Find the facilities with their usage by member, but not guests

In [8]:
# Q12: Find the facilities with their usage by member, but not guests
query = """
SELECT 
    f.name AS facility_name, 
    COUNT(b.bookid) AS usage_count
FROM Bookings AS b
INNER JOIN Facilities AS f ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name
ORDER BY usage_count DESC;
"""

df_q12 = pd.read_sql_query(query, con)
print(df_q12)


     facility_name  usage_count
0       Pool Table          783
1    Snooker Table          421
2   Massage Room 1          421
3     Table Tennis          385
4  Badminton Court          344
5   Tennis Court 1          308
6   Tennis Court 2          276
7     Squash Court          195
8   Massage Room 2           27


### Q13: Find the facilities usage by month, but not guests

In [9]:
# Q13: Find the facilities usage by month, but not guests
query = """
SELECT 
    f.name AS facility_name,
    STRFTIME('%m', b.starttime) AS month,
    SUM(b.slots) AS total_usage
FROM Bookings AS b
INNER JOIN Facilities AS f ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY facility_name, month
ORDER BY facility_name, month;
"""

df_q13 = pd.read_sql_query(query, con)
print(df_q13)

      facility_name month  total_usage
0   Badminton Court    07          165
1   Badminton Court    08          414
2   Badminton Court    09          507
3    Massage Room 1    07          166
4    Massage Room 1    08          316
5    Massage Room 1    09          402
6    Massage Room 2    07            8
7    Massage Room 2    08           18
8    Massage Room 2    09           28
9        Pool Table    07          110
10       Pool Table    08          303
11       Pool Table    09          443
12    Snooker Table    07          140
13    Snooker Table    08          316
14    Snooker Table    09          404
15     Squash Court    07           50
16     Squash Court    08          184
17     Squash Court    09          184
18     Table Tennis    07           98
19     Table Tennis    08          296
20     Table Tennis    09          400
21   Tennis Court 1    07          201
22   Tennis Court 1    08          339
23   Tennis Court 1    09          417
24   Tennis Court 2    07